<a href="https://colab.research.google.com/github/HemantharajM/Reinforcement-Learning-Intro-to-expect/blob/master/Multi_agent_planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import autograd
from torch.utils.data import DataLoader
from torch.nn.parameter import Parameter

import torch.nn.functional as F

In [4]:
!ls

sample_data


In [6]:
from google.colab import files
uploaded = files.upload()

Saving Env_28x28.npz to Env_28x28.npz
Saving Env_32x32.npz to Env_32x32.npz
Saving Env_64x64.npz to Env_64x64.npz
Saving Env_80x80.npz to Env_80x80.npz
Saving Env_128x128.npz to Env_128x128.npz


In [0]:
def action(S,val):
    if val == 0:
        return torch.tensor([S[0].item(),S[1].item()-1])
    elif val == 1:
        return torch.tensor([S[0].item()-1,S[1].item()])
    elif val == 2:
        return torch.tensor([S[0].item(),S[1].item()+1])
    elif val == 3:
        return torch.tensor([S[0].item()+1,S[1].item()])
    else :
        return S

In [0]:
def rand_play(X,a_s,imsize):
    #find possible action
    Action = []
    
    if (a_s[1] -1) >= 0:
        if X[0][a_s[0]][a_s[1]-1] and (not X[2][a_s[0]][a_s[1]-1]):
            Action.append(0)
    if (a_s[0] -1 ) >= 0:
        if X[0][a_s[0]-1][a_s[1]] and (not X[2][a_s[0]-1][a_s[1]]):
            Action.append(1)
    if (a_s[1] +1 ) < imsize:
        if X[0][a_s[0]][a_s[1]+1] and (not X[2][a_s[0]][a_s[1]+1]):
            Action.append(2)
    if (a_s[0] + 1) < imsize:    
        if X[0][a_s[0]+1][a_s[1]] and (not X[2][a_s[0]+1][a_s[1]]):
            Action.append(3)
    
    if len(Action) != 0:
        act = np.random.choice(Action)
        new_state = action(a_s,act)
    else:
        new_state = a_s
    
    return new_state

In [0]:
def goal_reach_check(agent_state,goal,goal_count,goal_reached):
    token = 0
    v = 0
    for z in goal:
        #check goal is already reached
        if goal_reached[v] == 0:
            #if goal is not reached and compare with agent state
            if agent_state[0].item() == z[0] and agent_state[1].item() == z[1]:
                goal_reached[v] = 1
                goal_count += 1
                token = 1
        v = v+1    
    return token,goal_count,goal_reached

In [0]:
def agent_predict(model,dom,goal,imsize,max_random_play):
    
    X = dom.clone()
    model.eval() 
    correct_goal,total_goal,traj_step_error,goal_count = 0.0,0.0,0.0,0.0
    
    #store agent initial state
    inter = np.where(X[2] == 1.0)
    agent_state = np.column_stack([inter[0],inter[1]])
    n_agent = agent_state.shape[0]
    
    agent_traj = [[agent_state[i].copy()] for i in range(n_agent)]
    
    #store number agent still playing
    current_play_agent = [1.0 for i in range(n_agent)]
    
    current_play_agent = np.array(current_play_agent)
    
    #keep which is reached
    goal_reached = np.zeros([goal.size])
    
    #store each agent random play chances
    random_play = np.zeros(n_agent)
    max_random_play = max_random_play
    

    #run untill all player finish
    while current_play_agent.sum() != 0.0: 
        for i in range(n_agent):
            
            if current_play_agent[i] == 1.0:    
                S1 = torch.tensor(agent_state[i][0])
                S2 = torch.tensor(agent_state[i][1])
                S1 = S1.to(device)
                S2 = S2.to(device)
                Input = X.clone()
                Input = Input.reshape([1,3,imsize,imsize])
                Input = Input.float()
                
                Input = Input.to(device)
                
                output,prediction = model(Input,S1,S2)
                _,index = torch.max(output,dim=1)
                #check agent (index = 4) means stop planning
                if index == 4:
                    #token whether agent reach goal or not
                    token,goal_count,goal_reached = goal_reach_check(agent_state[i],goal,goal_count,goal_reached)
                    if token:
                        current_play_agent[i] = 0.0
                    else:
                        if random_play[i] > max_random_play:
                            current_play_agent[i] = 0.0
                        else:
                            X[2][agent_state[i][0]][agent_state[i][1]] = 0.0
                            agent_state[i] =rand_play(X,agent_state[i].copy(),imsize)
                            random_play[i] += 1
                            X[2][agent_state[i][0]][agent_state[i][1]] = 1.0
                            agent_traj[i].append(agent_state[i].copy())
    
                else:
                    X[2][agent_state[i][0]][agent_state[i][1]] = 0.0
                    agent_state[i] = action(agent_state[i],index)
                    
                    #check agent reach any obstacle and push it into stop
                    i_1 = agent_state[i][0]
                    i_2 = agent_state[i][1]
                    if X[0][i_1][i_2].item() == 0.0 and X[1][i_1][i_2].item() == 0.0:
                        current_play_agent[i] = 0.0
                        
                    X[2][agent_state[i][0]][agent_state[i][1]] = 1.0
                    agent_traj[i].append(agent_state[i].copy())
                    
                    #stop agent which moves back and forward two on the goal
                    if len(agent_traj[i]) > 3 and (agent_traj[i][-1] == agent_traj[i][-3]).sum() == 2 and (agent_traj[i][-2] == agent_traj[i][-4]).sum() == 2:
                        
                        inter = agent_traj[i][-1]
                        token,goal_count,goal_reached = goal_reach_check(inter,goal,goal_count,goal_reached)
                        if token:
                            current_play_agent[i] = 0.0
                        else:     
                            inter = agent_traj[i][-2]
                            token,goal_count,goal_reached = goal_reach_check(inter,goal,goal_count,goal_reached)
                            if token:
                                current_play_agent[i] = 0.0
                                agent_traj[i].pop(-1)
                            else:
                                if random_play[i] < max_random_play:
                                    X[2][agent_state[i][0]][agent_state[i][1]] = 0.0
                                    agent_state[i] =rand_play(X,agent_state[i].copy(),imsize)
                                    random_play[i] += 1
                                    X[2][agent_state[i][0]][agent_state[i][1]] = 1.0
                                    agent_traj[i].append(agent_state[i].copy())
                                else:
                                    current_play_agent[i] = 0.0
                    
    
    correct_goal = goal_count
    
    if n_agent > len(goal):
        total_goal = len(goal)
    else:
        total_goal =  n_agent
    
    return correct_goal,total_goal,agent_traj

In [0]:
class MA_VIN(torch.nn.Module):
    def __init__(self, k,imsize):
        super(MA_VIN, self).__init__()
        self.k = k
        self.h = torch.nn.Conv2d(
            in_channels=3,
            out_channels=150,
            kernel_size=(3, 3),
            stride=1,
            padding=1,
            bias=True)
        self.r = torch.nn.Conv2d(
            in_channels=150,
            out_channels=1,
            kernel_size=(1, 1),
            stride=1,
            padding=0,
            bias=False)
        self.q = torch.nn.Conv2d(
            in_channels=1,
            out_channels=5,
            kernel_size=(3, 3),
            stride=1,
            padding=1,
            bias=False)
        self.fc_1 = torch.nn.Linear(in_features=5, out_features=5, bias=True)
        self.w = Parameter(torch.zeros(5, 1, 3, 3), requires_grad=True)
        self.sm = torch.nn.Softmax(dim=1)

    def forward(self, X, S1, S2):
        h = self.h(X)
        r = self.r(h)
        q = self.q(r)
        v, _ = torch.max(q, dim=1, keepdim=True)
        for i in range(0, self.k - 1):
            q = F.conv2d(
                torch.cat([r, v], 1),
                torch.cat([self.q.weight, self.w], 1),
                stride=1,
                padding=1)
            v, _ = torch.max(q, dim=1, keepdim=True)

        q = F.conv2d(
            torch.cat([r, v], 1),
            torch.cat([self.q.weight, self.w], 1),
            stride=1,
            padding=1)

        slice_s1 = S1.long().expand(imsize, 1, 5, q.size(0))
        slice_s1 = slice_s1.permute(3, 2, 1, 0)
        q_out = q.gather(2, slice_s1).squeeze(2)

        slice_s2 = S2.long().expand(1, 5, q.size(0))
        slice_s2 = slice_s2.permute(2, 1, 0)
        q_out = q_out.gather(2, slice_s2).squeeze(2)

        logits = self.fc_1(q_out)
        
        return logits, self.sm(logits)

In [0]:
test = []
with np.load('Env_28x28.npz', mmap_mode='r') as f:
    test = f['arr_0']


In [0]:
test_data = []

for i in test:
    env = -1 *(i[0] == 0) + 1.0
    goal = (i[0] == 0.5) + 0.0
    Agent = i[1]
    domain = np.array([env,goal,Agent])
    
    test_data.append([torch.tensor(domain.copy())])
    

In [30]:
model_28x28 = MA_VIN(60,28)
model_28x28.load_state_dict(torch.load('MA_VIN_6.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [31]:
model_28x28.cuda()

MA_VIN(
  (h): Conv2d(3, 150, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (r): Conv2d(150, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (q): Conv2d(1, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (fc_1): Linear(in_features=5, out_features=5, bias=True)
  (sm): Softmax()
)

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "CPU")

In [46]:
correct_goal,total_goal = 0.0,0.0   
imsize = 28
max_rand_play= 5
for i in range(50):
    X = test_data[i][0].clone()
    
    go = np.where(X[1] == 1.0)
    goal = np.column_stack([go[0],go[1]])
    
        
    current_goal,current_total,traj = agent_predict(model_28x28,X.clone(),goal,imsize,max_rand_play)
    
    correct_goal += current_goal
    total_goal += current_total
    
print(f'Goal Accuracy: {correct_goal/total_goal}')
print(correct_goal,total_goal)

Goal Accuracy: 0.7053571428571429
158.0 224.0


In [0]:
test = []
with np.load('Env_64x64.npz', mmap_mode='r') as f:
    test = f['arr_0']

In [0]:
test_data = []

for i in test:
    env = -1 *(i[0] == 0) + 1.0
    goal = (i[0] == 0.5) + 0.0
    Agent = i[1]
    domain = np.array([env,goal,Agent])
    
    test_data.append([torch.tensor(domain.copy())])

In [72]:
model_64x64 = MA_VIN(125,64)
model_64x64.load_state_dict(torch.load('MA_VIN_6.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [73]:
model_64x64.cuda()

MA_VIN(
  (h): Conv2d(3, 150, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (r): Conv2d(150, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (q): Conv2d(1, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (fc_1): Linear(in_features=5, out_features=5, bias=True)
  (sm): Softmax()
)

In [74]:
correct_goal,total_goal = 0.0,0.0   
imsize = 64
max_rand_play= 5
for i in range(50):
    X = test_data[i][0].clone()
    
    go = np.where(X[1] == 1.0)
    goal = np.column_stack([go[0],go[1]])
    
        
    current_goal,current_total,traj = agent_predict(model_64x64,X.clone(),goal,imsize,max_rand_play)
    
    correct_goal += current_goal
    total_goal += current_total
    
print(f'Goal Accuracy: {correct_goal/total_goal}')
print(correct_goal,total_goal)

Goal Accuracy: 0.5185185185185185
112.0 216.0


In [0]:
test = []
with np.load('Env_80x80.npz', mmap_mode='r') as f:
    test = f['arr_0']

In [0]:
test_data = []

for i in test:
    env = -1 *(i[0] == 0) + 1.0
    goal = (i[0] == 0.5) + 0.0
    Agent = i[1]
    domain = np.array([env,goal,Agent])
    
    test_data.append([torch.tensor(domain.copy())])

In [88]:
model_80x80 = MA_VIN(135,80)
model_80x80.load_state_dict(torch.load('MA_VIN_6.pth'))
model_80x80.cuda()

MA_VIN(
  (h): Conv2d(3, 150, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (r): Conv2d(150, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (q): Conv2d(1, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (fc_1): Linear(in_features=5, out_features=5, bias=True)
  (sm): Softmax()
)

In [89]:
correct_goal,total_goal = 0.0,0.0   
imsize = 80
max_rand_play= 5
for i in range(50):
    X = test_data[i][0].clone()
    
    go = np.where(X[1] == 1.0)
    goal = np.column_stack([go[0],go[1]])
    
        
    current_goal,current_total,traj = agent_predict(model_80x80,X.clone(),goal,imsize,max_rand_play)
    
    correct_goal += current_goal
    total_goal += current_total
    
print(f'Goal Accuracy: {correct_goal/total_goal}')
print(correct_goal,total_goal)

Goal Accuracy: 0.49324324324324326
146.0 296.0


In [0]:
test = []
with np.load('Env_128x128.npz', mmap_mode='r') as f:
    test = f['arr_0']

In [0]:
test_data = []

for i in test:
    env = -1 *(i[0] == 0) + 1.0
    goal = (i[0] == 0.5) + 0.0
    Agent = i[1]
    domain = np.array([env,goal,Agent])
    
    test_data.append([torch.tensor(domain.copy())])

In [98]:
model_128x128 = MA_VIN(190,128)
model_128x128.load_state_dict(torch.load('MA_VIN_6.pth'))
model_128x128.cuda()

MA_VIN(
  (h): Conv2d(3, 150, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (r): Conv2d(150, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (q): Conv2d(1, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (fc_1): Linear(in_features=5, out_features=5, bias=True)
  (sm): Softmax()
)

In [99]:
correct_goal,total_goal = 0.0,0.0   
imsize = 128
max_rand_play= 5
for i in range(50):
    X = test_data[i][0].clone()
    
    go = np.where(X[1] == 1.0)
    goal = np.column_stack([go[0],go[1]])
    
        
    current_goal,current_total,traj = agent_predict(model_128x128,X.clone(),goal,imsize,max_rand_play)
    
    correct_goal += current_goal
    total_goal += current_total
    
print(f'Goal Accuracy: {correct_goal/total_goal}')
print(correct_goal,total_goal)

Goal Accuracy: 0.34814814814814815
141.0 405.0
